In [1]:
# import libraries

import os

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings('ignore')

In [2]:
os.chdir("res")

In [3]:
# load dataset
df = pd.read_csv('kidney_disease.csv')
df.drop(columns=['id'],axis=1,inplace=True)

In [5]:
df.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification'],
      dtype='object')

In [6]:
df.columns = ['age', 'blood_pressure', 'spesific_gravity', 'albumin', 'sugar',
              'red_blood_cell', 'puss_cell', 'puss_cell_clumbs', 'bacteria', 
              'blood_glucose_random', 'blood_urea','serum_creatine', 'sodium',
              'potassium', 'hemoglobin', 'packet_cell_volume', 'white_blood_cell_count',
              'red_blood_cell_count', 'hyper_tension', 'diabetes_mellitus',
              'coronary_artert_disease','appetite', 'peda_edema', 'anemia', 'class']

In [7]:
df.head()

,age,blood_pressure,spesific_gravity,albumin,sugar,red_blood_cell,puss_cell,puss_cell_clumbs,bacteria,blood_glucose_random,...,packet_cell_volume,white_blood_cell_count,red_blood_cell_count,hyper_tension,diabetes_mellitus,coronary_artert_disease,appetite,peda_edema,anemia,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      391 non-null    float64
 1   blood_pressure           388 non-null    float64
 2   spesific_gravity         353 non-null    float64
 3   albumin                  354 non-null    float64
 4   sugar                    351 non-null    float64
 5   red_blood_cell           248 non-null    object 
 6   puss_cell                335 non-null    object 
 7   puss_cell_clumbs         396 non-null    object 
 8   bacteria                 396 non-null    object 
 9   blood_glucose_random     356 non-null    float64
 10  blood_urea               381 non-null    float64
 11  serum_creatine           383 non-null    float64
 12  sodium                   313 non-null    float64
 13  potassium                312 non-null    float64
 14  hemoglobin               3

In [10]:
def convert2numeric(col):
    df[col] = pd.to_numeric(df[col], errors='coerce')
    # coerce : if can't convert, value to nan

to_numeric_list = ['packet_cell_volume','white_blood_cell_count','red_blood_cell_count']

for col in to_numeric_list:
    convert2numeric(col)